# Environment Setup

This is a brief tutorial of how to setup the enviroment required to execute MOT tests using python and other libraries
for the thesis work of MOT from non frontal view in crowded scenes.

## 1. Installing Anaconda

It is necessary a source package managament system and an environment managament system to handle all the libraries and dependencies of our project. In this way the setup process is more simplified and user-friendly for others. Anaconda is the facto python SPM and EMS, to install it you can follow the steps described in their web page (take a look at the link below).

* **https://docs.conda.io/projects/conda/en/latest/user-guide/install/**

### 1.1. Creating an environment in Anaconda

Once you have installed it, just open the anaconda command shell and type in:

* **conda create --name thesis python=3**

This will create a new environment called thesis based on python 3. 
Once you have done that you can check if your environment was created successfully with the following command:

* **conda env list**

This command shows the current available environments in the system

### 1.2. Adding libraries in your environment

Now that you have created your environment you have to install all the libraries needed by the project. 
But first you have to open your environment, you can do that with the following command:

* **activate thesis**

And to install the libraries type in:

* **conda install numpy pandas matplotlib opencv scikit-learn jupyter notebook**

That command will install multiple libraries like: opencv, pandas, numpy, and other required by our project. To verify that the libraries were installed correctly you can type in the following command which displayies the installed libraries in the system:

* **conda list**

### 1.3. Adding python MOT metrics library

For this project we will use py-motmetrics library which source code is available in github and has a clean interface.
The py-motmetrics library can be found in the following link:

* **https://github.com/cheind/py-motmetrics**

To install it, first you have to download the code with git (git must be installed in your system to do the next steps).
Then just open a git bash command prompt and type in:

* **git clone https://github.com/cheind/py-motmetrics.git**

This will download the py-motmetrics source code in your computer. Once you have done that, open your environment in anaconda and navigate to py-motmetrics folder. Once there, type in the following commands:

* **conda install --file requirements.txt**
* **pip install .**
* **pytest**


## 2. Environment test

### 2.1. OpenCV test

The following code checks if opencv was correctly installed and it uses the MatchAndMingle dataset.
You have to modify the file_path and file_name of the code below to specify the correct one in your computer
to be able to run the code in your computer.

In [3]:
import cv2 as cv

# File path where your downsample vides are located and the file name of the video must be displayed
# UPDATE this with the correct file path and file name of MatchNMingle of your computer
file_path = 'C:/Master/Mingle/videos/downsample/'
file_name = '30min_day1_cam1_20fps_960x540.mp4'

caption = cv.VideoCapture(file_path + file_name)

# Position where a rectangle will be drawn
pts1 = (100,100)
pts2 = (200,200)

# Just loop for each of the frames in the vide if key "q" is pressed then it exits the execution
while caption.isOpened():
    ret, frame = caption.read()
    
    if not ret:
        print("Can't receive frame, error while reading, exiting ...")
        break
    
    cv.rectangle(frame, pt1=pts1, pt2=pts2, color=(0, 255, 0), thickness=3)
    cv.imshow('Frame', frame)
    
    if cv.waitKey(1) == ord('q'):
        break

# Release and destroy the window
caption.release()
cv.destroyAllWindows()

### 2.2. py-motmetrics test

This code below uses the py-motmetrics library, where 3 simple frames are logged using the MOTAccumulator API, at the end the MOTA and MOTP results are shown.

In [4]:
import motmetrics as mm
import numpy as np

# Create an accumulator that will be updated during each frame
acc = mm.MOTAccumulator(auto_id=True)

# Call update once for per frame. For now, assume distances between
# frame objects / hypotheses are given.
acc.update(
    [1, 2],                     # Ground truth objects in this frame
    [1, 2, 3],                  # Detector hypotheses in this frame
    [
        [0.1, np.nan, 0.3],     # Distances from object 1 to hypotheses 1, 2, 3
        [0.5, 0.2   , 0.3]      # Distances from object 2 to hypotheses 1, 2, 3
    ]
)
#print(acc.events)     # a pandas DataFrame containing all events
print(acc.mot_events) # a pandas DataFrame containing MOT only events
print()

frameid = acc.update(
    [1, 2],
    [1],
    [
        [0.2],
        [0.4]
    ]
)
print(acc.mot_events.loc[frameid])
print()

frameid = acc.update(
    [1, 2],
    [1, 3],
    [
        [0.6, 0.2],
        [0.1, 0.6]
    ]
)
print(acc.mot_events.loc[frameid])
print()

mh = mm.metrics.create()
summary = mh.compute(acc, metrics=['num_frames', 'mota', 'motp'], name='acc')
print(summary)

                Type  OId  HId    D
FrameId Event                      
0       6      MATCH  1.0  1.0  0.1
        7      MATCH  2.0  2.0  0.2
        8         FP  NaN  3.0  NaN

        Type  OId  HId    D
Event                      
3      MATCH  1.0  1.0  0.2
4       MISS  2.0  NaN  NaN

         Type  OId  HId    D
Event                       
5       MATCH  1.0  1.0  0.6
6      ASCEND  2.0  3.0  0.6
7      SWITCH  2.0  3.0  0.6

     num_frames  mota  motp
acc           3   0.5  0.34
